## Dataset: F1 Database from ERGAST (Regression Problem) 

## Table of Contents
#### 1. [Background](#part1)
#### 2. [Problem Statement](#part2)
#### 3. [Project Objectives](#part3)
#### 4. [Primary questions](#part4)
#### 5. [Datasets](#part5) 
#### 6. [Step 1: Load and Explore the data](#part6)  
   - [6.1 Importing Data with Pandas](#part6.1)
   - [6.2 Extracting year from unstructured date fields](#part6.2)
   - [6.3 Extracting year from unstructured text data](#part6.3)

#### 7. [Step 2: Data Wrangling on multiple tables](#part7)
   - [7.1 Merging different tables together](#part7.1)
   - [7.2 Feature Engineering using Mathematical Calculations](#part7.2) 
   - [7.3 Data Exploration and Visualization (Exploratory Data Analysis - EDA)](#part7.3)
   
#### 8. [Step 3: Data Cleansing and Transformation](#part8)
   - [8.1 Descriptive Statistics](#part8.1)
   - [8.2 Correlation](#part8.2)
   - [8.3 Univariate Analysis - Numerical variables](#part8.3)

## Background <a id="part1"></a>
Ergast.com is a webservice that provides a database of Formula 1 races, starting from the 1950 season until today. 

### Source & Acknowledgements

The dataset, downloaded from http://ergast.com/mrd/db/#csv, includes information such as the time taken in each lap, the time taken for pit stops, the performance in the qualifying rounds etc. of all Formula 1 races.

## Problem Statement <a id="part2"></a>

How to implement a linear regression model for predicting number of points achieved by a driver in F1 race

## Project Objectives <a id="part3"></a>

To extract data from a database, explore the data and formulate a prediction problem

To create a tabular data table from multiple tables based on the formulated problem

To wrangle and prepare the data ready for modeling, use the prepared data to build and evaluate a simple machine learning model

To document the process, analysis, comparison and findings

## Primary questions <a id="part4"></a>

Who (driver) is the winner for each year's Formula One Championship?

Which team (constructor) is the winner for each year's Formula One Championship?

Who (driver) has the most Championship titles till date?

Who (driver) has the most number of pole positions (grid = 1) till date?

Who (driver) has the most race wins (position = 1) till date?

How does the different grid (pole positions) affect points achievement?

How does the different rankings of fastest laps affect points achievement?

Does race age (young or old) impact on points achievement?

Does location of race impact points achievement?

## Datasets Provided <a id="part5"></a>

1. circuits;
2. constructorResults;
3. constructorStandings;
4. constructors (used here);
5. driverStandings;
6. drivers (used here);
7. lapTimes;
8. pitStops;
9. qualifying;
10. races (used here);
11. results (used here);
12. seasons;
13. status (used here).

In [ ]:
# import the pacakges
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# to show all the columns of the dataframe in the notebeook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

### Step 1: Load and Explore the Data <a id="part6"></a>

### Importing Data with Pandas <a id="part6.1"></a>

In [ ]:
# 1st Dataset "constructors_mod"

df_constructor = pd.read_csv('./raw_data/constructors.csv')
display(df_constructor.head(10))

# Drop the columns 'url', 'constructorRef', 'nationality' which are not useful

df_constructor.drop(columns = ['url', 'constructorRef', 'nationality'], axis=1, inplace=True)
df_constructor

In [ ]:
# Basic summary of the dataset

print(df_constructor.shape) # 211 rows, 2 columns
print(df_constructor.size) # 422 observations in total
print(df_constructor.dtypes) # 1 categorical column and 1 numeric column

In [ ]:
# 2nd Dataset "results_mod"

df_result = pd.read_csv('./raw_data/results.csv')
df_result.head(10)

In [ ]:
# Basic summary of the dataset

print(df_result.shape) # 25,399 rows, 18 columns
print(df_result.size) # 457,182 observations in total
print(df_result.dtypes) # 9 categorical columns (wrong datatype because of \N: should be int64) and 9 numeric columns

### Extracting year from unstructured date fields <a id="part6.2"></a>

In [ ]:
# 3rd Dataset "drivers_mod"

df_driver = pd.read_csv('./raw_data/drivers.csv')
display(df_driver.head(10))

# Extract year in unstructured datetime data 'dob'  

df_driver['yob'] = pd.to_datetime(df_driver['dob']).dt.year

# Apply mathematical Calculations to Features such as addition of 'forename' and 'surname' with an underscore

df_driver['drivername'] = df_driver['forename'] + '_' + df_driver['surname']

# Drop the column 'url', 'forename', 'surname', 'nationality', 'dob', 'driverRef' which are no longer useful

df_driver.drop(columns = ['url', 'code', 'forename', 'surname', 'nationality', 'dob', 'driverRef'], axis=1, inplace=True)
df_driver

In [ ]:
# Basic summary of the dataset

print(df_driver.shape) # 854 rows, 4 columns
print(df_driver.size) # 3416 observations in total
print(df_driver.dtypes) # 2 categorical columns and 2 numeric columns

### Extracting year from unstructured text data <a id="part6.3"></a>

In [ ]:
# 4th Dataset "races_mod"

df_circuit = pd.read_csv('./raw_data/races.csv')
display(df_circuit.head(10))

# Extract year in unstructured data 'url text'

df_circuit['year'] = df_circuit['url'].str.extract('(\d+)')

df_circuit

In [ ]:
# Display all the rows that have Null values

display(df_circuit[pd.isnull(df_circuit).any(axis=1)])

# Replace NaN value with 2021 in the year column

df_circuit['year'] = df_circuit['year'].replace(np.nan, 2021)

# Check to see if the new value 2021 is in index 1055

df_circuit.loc[1055]

In [ ]:
# Display the row where raceID = 1035. Notice that the year is 70 in the 'year' column which should not be the case

display(df_circuit.loc[df_circuit['raceId'] == 1035])

# Replace year 70 to be year 2020 and check again

df_circuit['year'] = df_circuit['year'].replace(['70'], '2020')
df_circuit.loc[1022]

In [ ]:
# convert the year column datatype to integer

df_circuit['year'] = df_circuit['year'].astype(int)

# Drop the column 'url', 'time', 'date' which are no longer useful

df_circuit.drop(columns = ['url', 'time', 'date'], axis=1, inplace=True)
df_circuit

In [ ]:
# Basic summary of the dataset

print(df_circuit.shape) # 1,080 rows, 5 columns
print(df_circuit.size) # 5,400 observations in total
print(df_circuit.dtypes) # 1 categorical column and 4 numeric columns

In [ ]:
# 5th Dataset "status_mod"

df_status = pd.read_csv('./raw_data/status.csv')
df_status.head(10)

In [ ]:
# Basic summary of the dataset

print(df_status.shape) # 137 rows, 2 columns
print(df_status.size) # 274 observations in total
print(df_status.dtypes) # 1 categorical column and 1 numeric column

### Step 2: Data Wrangling on multiple tables <a id="part7"></a>

### Merging different tables together <a id="part7.1"></a>

In [ ]:
# joining df_result with df_driver by column driverID, using inner join

df_result1 = pd.merge(df_result, df_driver, on='driverId')

# joining df_result1 with df_circuit by column raceID, using inner join

df_result2 = pd.merge(df_result1, df_circuit, on='raceId')

# joining df_result2 with df_constructor by column constructorID, using inner join

df_result3 = pd.merge(df_result2, df_constructor, on='constructorId')

# joining df_result3 with df_status by column statusID, using inner join

df_race_finished = pd.merge(df_result3, df_status, on='statusId')
df_race_finished

In [ ]:
# Drop the columns 'resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'positionText', 'positionOrder', 'time', 'milliseconds', 'fastestLapTime', 'statusId', 'number_y' and 'circuitId' which are not useful

df_race_finished.drop(columns = ['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'positionText', 'positionOrder', 'time', 'milliseconds', 'fastestLapTime', 'statusId', 'number_y', 'circuitId'], axis=1, inplace=True)
df_race_finished

In [ ]:
# rename the columns accordingly
df_race_finished.rename(columns={'name_x': 'location', 'name_y': 'constructorname'}, inplace=True)
df_race_finished

### Feature Engineer 'race_age' column using Mathematical calculations  <a id="part7.2"></a>

In [ ]:
# Applying Mathematical substrations between features 'year' and 'yob' to derive the race_age of the driver

df_race_finished['race_age'] = df_race_finished['year'] - df_race_finished['yob']
df_race_finished.drop(columns = ['yob'], axis=1, inplace=True)
df_race_finished

In [ ]:
# Basic summary of the dataset

print(df_race_finished.shape) # 25,399 rows, 14 columns
print(df_race_finished.size) # 355,586 observations in total
print(df_race_finished.dtypes) # 8 categorical columns ('position', 'fastestLap', 'rank', 'fastestLapSpeed' 
                                                # variables incorrect, should be numerical) and 6 numeric columns

In [ ]:
# show the info of dataset

df_race_finished.info() # there are no columns with missing values 

### Data Exploration and Visualization (Exploratory Data Analysis - EDA)  <a id="part7.3"></a>

### Driver's Championship ranking by season

In [ ]:
# Create a user-defined function championship_driver(year) that shows the total points accumulated by each driver 
# in a given season. Based on the highest total points we can derive the winner of the championship in a particular year

def championship_driver(year):
    """returns the total points for a driver in a particular season"""
    # Create a column named "total_points" to sum up the number of points obtained by each driver
    total_points = df_race_finished[(df_race_finished['year']== year)].groupby('drivername')['points'].sum().reset_index()
    
    # Sort total_points in descending order
    total_points = total_points.sort_values(by=['points'], ascending=False)
    
    # Get top 20 results based on highest number of total points
    total_points = total_points[0:20]
    
    # Show top 20 drivers
    drivers = total_points.index
    total_points.plot(kind = 'bar', x='drivername', y='points', figsize=(12,6), title = f"Driver´s championship ranking of {year}")

In [ ]:
# choose the year to show the ranking between 2004 - 2021
championship_driver(2021) # Max Verstappen is the winner

### Constructor's Championship ranking by season

In [ ]:
# Create a user-defined function championship_constructor(year) that shows the total points accumulated by each driver 
# in a given season. Based on the highest total points we can derive the winner of the championship in a particular year.

def championship_constructor(year):
    """returns the total points for a constructor in a particular season"""
    # Create a column named "total_points" to sum up the number of points obtained by each constructor
    total_points = df_race_finished[(df_race_finished['year']== year)].groupby('constructorname')['points'].sum().reset_index()
    
    # Sort total_points in descending order
    total_points = total_points.sort_values(by=['points'],ascending=False)
    
    # Get top 10 results based on highest number of total points
    total_points = total_points[0:10]
    
    # Get top 10 constructors
    drivers = total_points.index
    total_points.plot(kind = 'bar', x='constructorname', y='points', figsize=(12,6), title = f"Constructor´s championship ranking of {year}")

In [ ]:
# choose the year to show the ranking between 2004 - 2021
championship_constructor(2021) # Mercedes is the winner

### Drivers with most Championship titles: Lewis Hamilton (7) and Michael Schumacher (7)

In [ ]:
# Create a column named "sum_driver" to sum up the number of points obtained by each driver in each year

sum_driver = df_race_finished.groupby(['year','drivername'])['points'].sum().reset_index()

# Find the maximum number of points obtained by each driver and assign it to a new variable 'champions' for each year

champions = sum_driver.loc[sum_driver.reset_index().groupby(['year'])['points'].idxmax()]

# Count the frequency in which the driver obtain the maximum points for each year 

champions = champions['drivername'].value_counts().reset_index()

# Rename the columns as 'driver' and 'titles'

champions.rename(columns={'index':'drivername','count':'titles'}, inplace = True)

# Get top 20 results based on maximum points

champions = champions[0:20]

In [ ]:
# Plot a bar chart to show the drivers with highest number of wins in the F1 championship over the years
champions.plot(kind = 'bar', x='drivername', y='titles', figsize=(12,6), title = f"Drivers with more Championship titles")

### Driver with most pole positions (grid = 1): Lewis Hamilton (103)

In [ ]:
# Create a column named "driver_pole" to sum up the number of pole positions (grid) obtained by each driver

driver_pole = df_race_finished[df_race_finished['grid'] == 1].groupby('drivername')['grid'].sum().reset_index()

# Sort driver_pole in descending order

driver_pole = driver_pole.sort_values(by=['grid'], ascending=False)

# Get top 20 results based on total pole positions obtained by each driver

driver_pole = driver_pole[0:20]

In [ ]:
# Plot a bar chart to show the drivers with highest number of pole positions (grid) in the F1 championship over the years

driver_pole.plot(kind = 'bar', x='drivername', y='grid', figsize=(12,6), title = f"Drivers with more pole positions (grid = 1)")

### Driver with most numbers of fastest lap (rank = 1) : Lewis Hamilton (59)

In [ ]:
# Convert the datatype of rank to integer 

df_race_finished['rank'] = pd.to_numeric(df_race_finished['rank'], errors='coerce')

# Subset the dataset by filtering for rank = 1 (fastest lap) and assign it to a new variable 'num_lap_rank'

num_lap_rank = df_race_finished[df_race_finished['rank'] == 1].groupby('drivername')['rank'].sum().reset_index()

# Sort num_lap_rank in descending order

num_lap_rank = num_lap_rank.sort_values(by=['rank'], ascending=False)

# Get top 20 results based on number of fastest lap obtained by each driver

num_lap_rank = num_lap_rank[0:20]

In [ ]:
# Plot a bar chart to show the top 20 drivers with highest number of fastest lap in F1 championship over the years

num_lap_rank.plot(kind = 'bar', x='drivername', y='rank', figsize=(12,6), title = f"Drivers with most numbers of fastest lap (rank = 1)")

### Driver with most race wins (position = 1): Michael Schumacher (91)

In [ ]:
# Subset the dataset by filtering for position = 1 (Wins) and assign it to a new variable 'num_wins'

num_wins = df_race_finished[(df_race_finished['position'] == '1')]

# Convert the datatype of position to integer 

num_wins['position'] = num_wins['position'].astype(int)

# Sum up the number of wins obtained by each driver

num_wins = num_wins.groupby('drivername')['position'].sum().reset_index()

# Sort num_wins in descending order

num_wins = num_wins.sort_values(by=['position'], ascending=False)

# Get top 20 results based on highest number of total points

num_wins = num_wins[0:20]

In [ ]:
# Plot a bar chart to show the top 20 drivers with highest number of wins in F1 championship over the years

num_wins.plot(kind = 'bar', x='drivername', y='position', figsize=(12,6), title = f"Drivers with more race wins (position = 1)")

### Step 3: Data Cleansing  <a id="part8"></a>

In [ ]:
# Note that there are some columns with '\\N' and NaN values that have caused their datatypes to be incorrect

df_race_finished

In [ ]:
# drop the date and time columns that have no meaningful contributions to modelling

df_race_finished = df_race_finished.drop(columns = ['fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', \
                                                    'quali_date', 'quali_time', 'sprint_date', 'sprint_time'])

In [ ]:
# check the remaining columns with '\\N' values 

for col in df_race_finished.columns:
    print(col)
    display(df_race_finished[df_race_finished[col] == '\\N'])
    print("==================================================")
    print()

In [ ]:
# replace the '\N' values with NaN values

df_race_finished.replace('\\N', np.NaN, inplace = True)

In [ ]:
# convert the columns with the correct datatype 

df_race_finished['position'] = df_race_finished['position'].astype("Int64")
df_race_finished['fastestLap'] = df_race_finished['fastestLap'].astype("Int64")
df_race_finished['rank'] = df_race_finished['rank'].astype("Int64")
df_race_finished['fastestLapSpeed'] = df_race_finished['fastestLapSpeed'].astype(float)

In [ ]:
df_race_finished.isnull().sum()

In [ ]:
# check that the selected columns' datatypes are corrected
df_race_finished.info()

###  Descriptive statistics  <a id="part8.1"></a>

In [ ]:
# Use descriptive statistics to check if there are anything amiss in each numerical column. 
# All variables seem ok at the moment (no negative values observed).

df_race_finished.describe()

###  Correlation  <a id="part8.2"></a>

#### grid, rank (Input variables) vs. points (Target variable) are highly negatively correlated to each other. Rank and grid, as well as laps and fastestLap are positively correlated to each other. 

In [ ]:
num_cols = [n for n in df_race_finished.columns if df_race_finished[n].dtypes != "O"]

# Use correlation to check if there are features that are highly correlated to each other

plt.title("Correlation of features in Formula One Dataset") # title name
sns.heatmap(df_race_finished[num_cols].corr(), cmap = 'coolwarm') # generate a heatmap using sns.heatmap(df_race_finished.corr(), cmap = 'coolwarm')

### Univariate analysis - numerical variables  <a id="part8.3"></a>

In [ ]:
# find all numerical variables

num_cols = [c for c in df_race_finished.columns if df_race_finished[c].dtypes !='O']
df_race_finished[num_cols].head()

In [ ]:
cat_cols = [c for c in df_race_finished.columns if df_race_finished[c].dtypes =='O']

In [ ]:
df_race_finished[num_cols].columns

In [ ]:
df_race_finished[cat_cols].columns

In [ ]:
# plot histograms of the num variables

df_race_finished[num_cols].hist(bins=30, figsize=(12,12))
plt.show()

In [ ]:
df_race_finished.info()

In [ ]:
for col in df_race_finished.columns:
    print(col)
    print(df_race_finished[col].unique())
    print("===========================")
    print()

In [ ]:
df_race_finished.to_csv("./processed_data/F1_race.csv", index=False)

## End of Notebook